# Getting Started with Segmentations

## In this notebook the following operations will be demonstrated:

1. Login and establish session to access the portal
2. List the datasets available on the portal
3. Upload a dataset
4. Access and download a dataset 
5. Delete a dataset


In [ ]:
import shapeworks as sw
from shutil import rmtree
from pathlib import Path

In [ ]:
from swcc.api import swcc_session
from swcc.models import Dataset
from swcc.models import (
    Dataset, GroomedSegmentation, OptimizedParticles,
    OptimizedShapeModel, Project, Segmentation, Subject
)

## Establish session and list the available datasets

In [ ]:
#api as a context manager
with swcc_session()  as session:
    token = session.login('username', 'password')
    session = swcc_session(token=token).__enter__()
    print([(d.id, d.name) for d in Dataset.list()])

In [ ]:
# For development, it is possible to generate a session outside of a context manager.
ctx = swcc_session(token=token)
session = ctx.__enter__()
print([(d.id, d.name) for d in Dataset.list()])

In [ ]:
# To connect to a different server, sepecify it as part of the session:
if False:  # Enable to run this cell
    base_url = 'http://burgess:8000/api/v1'
    with swcc_session(base_url=base_url) as session:
        token = session.login('admin@noemail.com', 'django-password')
    ctx = swcc_session(base_url=base_url, token=token)
    session = ctx.__enter__()

### You can list all entities on the server using the `list` method
Fields of dataset are:
1. id
2. name 
3. description 
4. license
5. acknowledgement
6. keywords
7. contributors
8. publications 

In [ ]:
for d in Dataset.list(): 
    print(d.name,d.id,d.description)
#     print(d)

## Upload a datset
In order to upload the a dataset you need:
1. Segmentations/Images/Meshes 
2. Data file (xlxs) 
- The sheet has to be named -'data'
- That sheet has to contain two columns: shape_file and image_file, both of which contain corresponding lists of relative paths to .nrrd files.
   
### <font color='red'>Try data sheet with mesh files and what can be done when a dataset has meshes and segmentations</b></i>

3. Project file (xlxs)- Any number of projects can be aded to a dataset. 
- Each project consists of an Excel .xlsx file with two sheets: data and optimize. 
- The data sheet has columns shape_file, groomed_file, alignment_file, local_particles_file, and world_particles_file. The shape file and groomed file columns contain relative paths to .nrrd files. 
- The local and world particles file columns contain relative paths to .particles files. 
- The alignment file column is a 12 parameter transform consisting of numbers separated by spaces. 
- The optimize sheet has columnes key and value which contain model parameters.

### <font color='red'> Project file generated from studio has more then two sheets.
    - data - explained above
    - groom - grooming parameter values
    - optimize - explained above
    - analysis - reconstruction algorithm parameters
    - deepssm - deepssm parameter values
    - studio - studio view options
    - project - 
    


In [ ]:
# All domain models are pydantic models that can be created in offline mode:
dataset = Dataset(name='test dataset2', license='license', description='just a test', acknowledgement='NIH')

# Note that it is missing an `id`.
dataset

In [ ]:
# To generate the entity on the server, you must call its `create` method
dataset.create()
# Now its id exists
dataset

In [ ]:
# Given an id, you can fetch data from the server
dataset_id = dataset.id
Dataset.from_id(dataset_id)


### <font color='red'>Can we have an option which can specify, that an existing instance of the dataset can be deleted and replaced with the new one being uploaded? 
</font>

In [ ]:

old = Dataset.from_name('ellipsoid_test')
# Either get rid of an existing dataset with the same name, or create a new name for testing

if old:
  print('Deleting previous version of %s' % 'ellipsoid_test')
  old.delete()
    

In [ ]:
dataset = Dataset(
        name='ellipsoid_test',
        license='Users should acknowledge that the collection and processing of these images was supported by the National Institutes of Health',
        description='Unaligned ellipsoids that vary along the x-axis. \
                     These ellipsoids have randomized size, center, rotation, and x radii.\
                     Images and segmentation have spacing [1,1,2]',
        acknowledgement='NIH')

dataset.create()
data = dataset.load_data_spreadsheet(Path('../Output/ellipsoid/data.xlsx'))
project = dataset.add_project(Path('../Output/ellipsoid/project.xlsx'))


## We can also explore the data using the models

### <font color='red'>How to add the anatomy type? 

In [ ]:
segmentation = next(data.segmentations)
print(f'Anatomy type: {segmentation.anatomy_type}') 

### <font color='red'>and how to address the other sheets from the project file?

In [ ]:
shape_models = next(project.shape_models)
shape_models.parameters

Right now, we can access the shape parameters from the project file

In [ ]:
#access individual parameters of shape model
print("Number of particles", shape_models.parameters["number_of_particles"])
print("Narrow band ", shape_models.parameters["narrow_band"])
print("Use normals ", shape_models.parameters["use_normals"])

## Given an ID, fetch data from the server

In [ ]:
print([(d.id, d.name) for d in Dataset.list()])

In [ ]:
dataset_id = 78
Dataset.from_id(dataset_id)

## How to access shape segmentations

### Each segmentation has the following attributes:
- id
- file 
- anatomy_type
- subject:
    - id 
    - name 
    - dataset
    

In [ ]:
dataset_name = 'ellipsoid_test'
# print(Dataset.from_name(dataset_name))
seg = next(Dataset.from_name(dataset_name).segmentations)
print(seg)

### <font color='red'> How to access shape files and images?

In [ ]:
dataset_name = 'ellipsoid_test'
print(Dataset.from_name(dataset_name).images)

## Delete a dataset

In [ ]:
# dataset = Dataset.from_name('ellipsoid8_3')
# dataset.delete()
# print([(d.id, d.name) for d in Dataset.list()])

## Download a dataset

In [ ]:
# Download a full dataset in bulk
dataset = Dataset.from_name('ellipsoid_test')
download_path = Path('../Output/downloads/ellipsoid/')
if download_path.exists():
    rmtree(str(download_path))
    
dataset.download(download_path)

## Not clear how to use this ? 
    
### <font color='red'>Here project variable was used when uploading the new ellipsoid dataset. When we are accessing the dataset from the portal, how can we access the project part? </font>

### <font color='red'> Do we always have to read the project files to access the shape models ? </font>

In [ ]:
new_dataset = Dataset.from_name(dataset_name)
new_dataset.Project
print(Dataset.from_name(dataset_name).project)

print('')

# There is also a helper method on models to download all files attached to them
groomed = next(project.groomed_segmentations)
for download in groomed.download_files('downloads/groomed'):
    print(f'Downloaded {download}...')

In [ ]:
# We can also explore the data using the models


shape_models = next(project.shape_models)
shape_models.parameters

dataset_name = 'ellipsoid_test'
# print(Dataset.from_name(dataset_name))
seg = next(Dataset.from_name(dataset_name).segmentations)
print(seg)

# The entities contain urls to download the associated files
print('Download url:')
print(seg.file.url)

print('')

# There is a helper method to download the file to a provided path
print(f"Downloaded {seg.file.download('downloads')}...")


### <font color='red'> How to access the urls for other files like images/particle data? 